### predicting if loans will be paid back
clean data: filter and deal with missing values <br>
try different prediction methods

In [ ]:
import pandas as pd
import numpy 


loans_2007=pd.read_csv('loans_2007.csv')
print(loans_2007.head(1),loans_2007.shape[1])

#remove columns that give away information about the future

loans_2007.drop(['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title','issue_d'],axis=1, inplace=True)

loans_2007=loans_2007.drop(['zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp'],axis=1)

loans_2007=loans_2007.drop(['total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt','last_pymnt_d'],axis=1)
#Removing single value columns ##

columns_to_drop=[]
for col in loans_2007.columns:
    #loans_2007[col]=loans_2007[col].dropna()
    if len(loans_2007[col].dropna().unique())==1:
        columns_to_drop.append(col)
loans_2007=loans_2007.drop(columns_to_drop,axis=1)

print(loans_2007.head(),loans_2007.shape[1])


#decide on the target column: loan_status
print(loans_2007['loan_status'].value_counts())

#goal: binary classification if loan will be paid back or not, reduction to two outcomes
loans_2007=loans_2007.loc[(loans_2007['loan_status']=='Fully Paid')|(loans_2007['loan_status']=='Charged Off')]
map_dict={'Fully Paid':1,
          'Charged Off':0}

loans_2007['loan_status']=loans_2007['loan_status'].replace(map_dict)
print(loans_2007['loan_status'].unique())

#result is filtered data
                          


        

In [ ]:
## further cleaning of data: handling missing values ##

#import pandas as pd
loans=loans_2007

#check amount of missing values

null_counts=loans.isnull().sum()
print_null_counts=null_counts[null_counts!=0]
print(print_null_counts)
print(type(null_counts))

## Handling missing values: drop column with mostly NaNs, drop rows with mostly NaNs: ##

loans=loans.drop('pub_rec_bankruptcies',axis=1)
loans=loans.dropna(axis=0)
check=loans.isnull().sum()
print(check[check>0])
print(loans.dtypes.value_counts())

## turn categorical columns into numerical columns##

object_columns_df=loans.select_dtypes(include=['object'])
print(object_columns_df.head(1))




## 7. Categorical columns ##

mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans=loans.replace(mapping_dict)

## title and purpose give same information, purpose has less unique values: drop title; also drop other columns that can't be easliy converted into numbers ##

print(loans['title'].value_counts())
print(loans['purpose'].value_counts())
loans=loans.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'],axis=1)

#int_Rate and revol_util are numerical if % is removed:

for l in ['int_rate', 'revol_util']: 
    loans[l]=(loans[l].str.rstrip('%')).astype('float')

## Dummy variables for few unique categorical values ##
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for col in cols:
    print(loans[col].value_counts())
    
dummy_df=pd.get_dummies(loans[['home_ownership','verification_status','purpose','term']])
loans=pd.concat([loans,dummy_df],axis=1)
loans=loans.drop(['home_ownership','verification_status','purpose','term'],axis=1)


## try different models : goal: conservative investor, get nearly all loans paid back
1 assume all loans are paid back <br>
2 Logistic Regression <br>
3 Random Forest Regressor

In [ ]:


print(loans.info())

## Picking an error metric ##


#tn=len(predictions[(predictions==0)&(loans['loan_status']==0)])
#tp=len(predictions[(predictions==1)&(loans['loan_status']==1)])
#fn=len(predictions[(predictions==0)&(loans['loan_status']==1)])
#fp=len(predictions[(predictions==1)&(loans['loan_status']==0)])


print(predictions.head(1))
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
                   

#realize Class imbalance - will be dealt with later

# 1. first SIMPLE model: Realize good accuracy when predicting that all loans will be paid off on time (imbalance!)
predictions = pd.Series(numpy.ones(loans.shape[0]))

tn=len(predictions[(predictions==0)&(loans['loan_status']==0)])
tp=len(predictions[(predictions==1)&(loans['loan_status']==1)])
fn=len(predictions[(predictions==0)&(loans['loan_status']==1)])
fp=len(predictions[(predictions==1)&(loans['loan_status']==0)])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

## 2. Logistic Regression simple model ##

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
features=loans.drop('loan_status', axis=1)
target=loans['loan_status']
lr.fit(features,target)
predictions=lr.predict(features)
#print(predictions.values_count())

tp=len(predictions[(predictions==1)&(target==1)])
fp=len(predictions[(predictions==1)&(target==0)])
tn=len(predictions[(predictions==0)&(target==0)])
fn=len(predictions[(predictions==0)&(target==1)])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print(fpr,tpr)

## 2a. Logistic Regression including Cross Validation ##

#from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression()
predictions=cross_val_predict(lr,features,target,cv=3)
predictions=pd.Series(predictions)

tp=len(predictions[(predictions==1)&(target==1)])
fp=len(predictions[(predictions==1)&(target==0)])
tn=len(predictions[(predictions==0)&(target==0)])
fn=len(predictions[(predictions==0)&(target==1)])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print(fpr,tpr)


## 2b address the imbalance: Penalizing the classifier ##

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression(class_weight='balanced')
predictions=cross_val_predict(lr,features,target,cv=3)
predictions=pd.Series(predictions)

tp=len(predictions[(predictions==1)&(target==1)])
fp=len(predictions[(predictions==1)&(target==0)])
tn=len(predictions[(predictions==0)&(target==0)])
fn=len(predictions[(predictions==0)&(target==1)])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print(fpr,tpr)

## 2c increase penalties manually##

#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import cross_val_predict

penalty={
    0:10,
    1:1
}

lr = LogisticRegression(class_weight=penalty)
predictions=cross_val_predict(lr,features,target,cv=3)
predictions=pd.Series(predictions)

tp=len(predictions[(predictions==1)&(target==1)])
fp=len(predictions[(predictions==1)&(target==0)])
tn=len(predictions[(predictions==0)&(target==0)])
fn=len(predictions[(predictions==0)&(target==1)])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print(fpr,tpr)


## 3 Model Random forests ##

from sklearn.ensemble import RandomForestClassifier
#from sklearn.cross_validation import cross_val_predict
penalty={
    0:10,
    1:1
}


rf = RandomForestClassifier(class_weight='balanced',random_state=1)
predictions=cross_val_predict(rf,features,target,cv=3)
predictions=pd.Series(predictions)

tp=len(predictions[(predictions==1)&(target==1)])
fp=len(predictions[(predictions==1)&(target==0)])
tn=len(predictions[(predictions==0)&(target==0)])
fn=len(predictions[(predictions==0)&(target==1)])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print(fpr,tpr)

## further tuning necessary
try ensemle of models, other models,...